In [1]:
from trajectory.models import Trajectory

qs = Trajectory.objects.all()
qs

<QuerySet [<Trajectory: default>, <Trajectory: default>]>

In [2]:
for t in qs:
    print(t.data)

{'2022': {'value': 100.0, 'updated': False}, '2023': {'value': 5.4, 'updated': False}, '2024': {'value': 5.4, 'updated': False}, '2025': {'value': 5.4, 'updated': False}, '2026': {'value': 5.4, 'updated': False}, '2027': {'value': 5.4, 'updated': False}, '2028': {'value': 5.4, 'updated': False}, '2029': {'value': 5.4, 'updated': False}, '2030': {'value': 5.4, 'updated': False}}
{'2022': {'value': 100.0, 'updated': True}, '2023': {'value': 5.4, 'updated': False}, '2024': {'value': 5.4, 'updated': False}, '2025': {'value': 5.4, 'updated': False}, '2026': {'value': 5.4, 'updated': False}, '2027': {'value': 5.4, 'updated': False}, '2028': {'value': 5.4, 'updated': False}, '2029': {'value': 5.4, 'updated': False}, '2030': {'value': 5.4, 'updated': False}}
